In [1]:
!pip install -q transformers datasets torch unsloth ipywidgets scikit-learn numpy plotly pandas hf_transfer
import os
os.environ['HF_HUB_ENABLE_HF_TRANSFER'] = '1'


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from datasets import load_dataset

dataset = load_dataset("RoBiology/RoBiologyDataChoiceQA")
dataset

DatasetDict({
    train: Dataset({
        features: ['question_number', 'question', 'type', 'options', 'grade', 'stage', 'year', 'right_answer', 'source', 'id_in_source', 'dupe_id'],
        num_rows: 11368
    })
    validation: Dataset({
        features: ['question_number', 'question', 'type', 'options', 'grade', 'stage', 'year', 'right_answer', 'source', 'id_in_source', 'dupe_id'],
        num_rows: 1376
    })
    test: Dataset({
        features: ['question_number', 'question', 'type', 'options', 'grade', 'stage', 'year', 'right_answer', 'source', 'id_in_source', 'dupe_id'],
        num_rows: 1388
    })
})

In [3]:
train_ds = dataset['train'].filter(lambda x: x['type'] == 'single-choice')
validation_ds = dataset['validation'].filter(lambda x: x['type'] == 'single-choice')

train_ds

Dataset({
    features: ['question_number', 'question', 'type', 'options', 'grade', 'stage', 'year', 'right_answer', 'source', 'id_in_source', 'dupe_id'],
    num_rows: 4860
})

In [4]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2046
dtype = None
load_in_4bit = False
load_in_8bit = False
random_state = 42

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    load_in_8bit = load_in_8bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.2: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.546 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [5]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = random_state,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.12.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [9]:
def create_chat_prompt(instruction: str, input: str, response: str|None = None):
    return tokenizer.apply_chat_template(
        [
            {"role": "system", "content": instruction},
            {"role": "user", "content": input},
            *([{"role": "assistant", "content": response}] if response else []),
        ],
        tokenize=False,
    )


In [10]:
instuction = "Answer the question based on the given options. Respond by writing only the letter of the correct answer."

print(create_chat_prompt(
    instuction,
    "\n".join([train_ds[0]["question"]] + train_ds[0]["options"]),
    train_ds[0]["right_answer"],
))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 07 Dec 2024

Answer the question based on the given options. Respond by writing only the letter of the correct answer.<|eot_id|><|start_header_id|>user<|end_header_id|>

Celula procariotă:
A. caracterizea ză virusurile, bacteriile și algele alba stre verzi
B. conține pept idioglican în compoziția membranei celulare
C. nu prezintă perete celular
D. materialul nuclear este o moleculă circulară de ADN bicatenar<|eot_id|><|start_header_id|>assistant<|end_header_id|>

D<|eot_id|>


In [11]:
train = train_ds.map(lambda x: {
    "chat": create_chat_prompt(instuction, '\n'.join([x['question']] + x['options']), x['right_answer']) 
})

validation = validation_ds.map(lambda x: {
    "chat": create_chat_prompt(instuction, '\n'.join([x['question']] + x['options']), x['right_answer']) 
})

Map:   0%|          | 0/4860 [00:00<?, ? examples/s]

Map:   0%|          | 0/582 [00:00<?, ? examples/s]

In [ ]:
try:
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

try:
    drive.mount('/content/drive')
    DRIVE_MOUNTED = True
except Exception:
    DRIVE_MOUNTED = False

if not IN_COLAB:
    final_model_path = './lora_model'
    checkpoints_path = './checkpoints'
else:
    if DRIVE_MOUNTED:
        final_model_path = '/content/drive/MyDrive/Colab Notebooks/finetuning/lora_model'
        checkpoints_path = '/content/drive/MyDrive/Colab Notebooks/finetuning/checkpoints'
    else:
        print("!" * 100)
        print("DRIVE NOT MOUNTED, all saved files will be deleted after the session ends")
        print("!" * 100)
        final_model_path = './lora_model'
        checkpoints_path = './checkpoints'

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train,
    dataset_text_field = "chat",
    eval_dataset=validation,
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        save_strategy = "steps",
        save_steps = 100,
        
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        # max_steps = 60, 
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = random_state,
        output_dir = checkpoints_path,
        report_to = "none", # Use this for WandB etc

        do_eval = True,
        eval_on_start = True,
        eval_strategy = "steps",
        eval_steps = 100,
    ),
)

Map (num_proc=2):   0%|          | 0/4860 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/582 [00:00<?, ? examples/s]

In [13]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.546 GB.
6.539 GB of memory reserved.


In [14]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 4,860 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,214
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss,Validation Loss
0,No log,3.655867
100,0.982300,1.266374
200,1.120200,1.203561
300,0.912500,1.162083
400,0.927200,1.132181
500,0.768000,1.108480
600,1.060400,1.088678
700,0.615900,1.084146
800,0.620200,1.068870
900,0.876300,1.069595


In [26]:
model.save_pretrained(final_model_path) # Local saving
tokenizer.save_pretrained(final_model_path)
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')